In [1]:

import keras
from keras.models import Model,Sequential
from keras.datasets import mnist
from tqdm import tqdm
# from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import pandas as pd
from PIL import Image
from skimage import color
from skimage import io
import skimage
import zipfile
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import LabelBinarizer 
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import random
from keras.callbacks import ModelCheckpoint 
from keras.utils import plot_model
from glob import glob
from keras.preprocessing.image import ImageDataGenerator
import os, sys
import shutil
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import  Dense, Dropout, LeakyReLU,  BatchNormalization, Conv2DTranspose
from tqdm import tqdm
from keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
%matplotlib inline

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from zipfile import ZipFile
with ZipFile('/content/drive/My Drive/Colab Notebooks/animefacedataset.zip', 'r') as zipObj:
   # Extract all the contents of zip file in the data directory
   zipObj.extractall('./data/')

In [10]:
DATA_FOLDER = '/content/data'
INPUT_DIM = (64,64,3) # Image dimension
BATCH_SIZE = 63565
Z_DIM = 200 # Dimension of the latent vector (z)
#Loading the data
batch_count = 63565 / 64

train_data = ImageDataGenerator(rescale=1./255).flow_from_directory(DATA_FOLDER, 
                                                                   target_size = INPUT_DIM[:2],
                                                                   batch_size = BATCH_SIZE,
                                                                   shuffle = True,
                                                                   class_mode = 'input',
                                                                   subset = 'training'
                                                                   )
data = next(train_data)
data = data[0]
batch_size=64

Found 63565 images belonging to 1 classes.


In [0]:
def Generator():
    generator=Sequential(name='Generator')
    generator.add(Dense(units=256,input_dim=25, name='gen_input'))
    generator.add(LeakyReLU(0.2))
    generator.add(BatchNormalization(momentum=0.8))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    generator.add(BatchNormalization(momentum=0.8))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    generator.add(BatchNormalization(momentum=0.8))
    
    generator.add(Dense(units=12288, activation='tanh', name='gen_output'))

    generator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
    return generator

In [0]:
def Discriminator():
    discriminator=Sequential(name='Discriminator')
    discriminator.add(Dense(units=512,input_dim=12288, name='disc_input'))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.2))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.2))
       
    discriminator.add(Dense(units=128))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=64))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Dense(units=1, activation='sigmoid', name='disc_output'))
    
    discriminator.compile(loss='binary_crossentropy',  optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
    return discriminator
    

In [9]:
generator=Generator()
generator.summary()
discriminator =Discriminator()
discriminator.summary()
discriminator.trainable=False
gan_input = Input(shape=(25,), name='GAN_input')
gan= Model(gan_input, discriminator(generator(gan_input)) , name='GAN_model')
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002,beta_1=0.5))
gan.summary()

Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gen_input (Dense)            (None, 256)               6656      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)               2048      
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              52

In [0]:


examples=25
dim=(5,5)
figsize=(30,30)
for e in range(1,6000+1 ):
    print("Epoch %d" %e)
    for _ in tqdm(range(batch_size)):
        noise= np.random.normal(0,1, [batch_size, 25])
        
        generated_images = generator.predict(noise)
        image_batch = []   
        for c in range(64):
            image_batch.append(data[np.random.randint(low=0,high=63565)])
        image_batch = np.array(image_batch)
        image_batch = image_batch.reshape(64,12288)
        X= np.concatenate((image_batch, generated_images))
        
        y_dis=np.zeros(2*batch_size)
        y_dis[:batch_size]=0.9

        discriminator.trainable=True
        discriminator.train_on_batch(X, y_dis)
        
        noise= np.random.normal(0,1, [batch_size, 25])
        y_gen = np.ones(batch_size)
      
        discriminator.trainable=False
        
        gan.train_on_batch(noise, y_gen)
        
    if e == 1 or e % 100 == 0:
        noise= np.random.normal(loc=0, scale=1, size=[examples, 25])
        generated_images = generator.predict(noise).reshape(25,64,64,3)
        plt.figure(figsize=figsize)
        for i in range(generated_images.shape[0]):
            plt.subplot(dim[0], dim[1], i+1)
            plt.imshow(generated_images[i], interpolation='nearest')
            plt.axis('off')
        plt.tight_layout()
        plt.savefig('/content/drive/My Drive/Colab Notebooks/gan_gen_image/gan_generated_image%d.png' %e)

  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1


100%|██████████| 64/64 [00:08<00:00,  7.88it/s]
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] 

Epoch 2


  3%|▎         | 2/64 [00:00<00:04, 13.51it/s]

...

Epoch 2110


  3%|▎         | 2/64 [00:00<00:04, 12.69it/s]

Epoch 2111


  9%|▉         | 6/64 [00:00<00:04, 12.57it/s]

Buffered data was truncated after reaching the output size limit.